In [2]:
import spacy
from spacy import displacy
import numpy as np
import pandas as pd
import random
import re
import operator
import math
import itertools
import json
from spacy.util import minibatch, compounding
from spacy.training import Example
from spacy.tokens import Doc
from pathlib import Path
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_md')


ner = nlp.get_pipe('ner')

# Load excel into pandas dataframe


In [3]:
training = pd.ExcelFile('NER training.xlsx')
df_training = training.parse('Sheet1')
df_training

Query  Intent  \
0              Time is 2030 and the date is 20/08/2030     NaN   
1                         Set a reminder at 1500 later     NaN   
2                                        Today at 0830     NaN   
3                                 Print report at 1900     NaN   
4    manufacturing facilitation efficiency of ab380...     NaN   
..                                                 ...     ...   
122              MTBF of EAGLE AR increase by how much     NaN   
123  which IDEALMOLD oee increase more than 20% fro...     NaN   
124                 number of ms90 tst less than 30min     NaN   
125          number of ps410 with tsdt more than 20min     NaN   
126         number of tlb139 with tpt more than 2hours     NaN   

                                    <ATTR>      <OBJ>       <TEMPORAL>  \
0                                      NaN        NaN  2030|20/08/2030   
1                                      NaN        NaN             1500   
2                                      NaN        NaN       0830|Today   
3                                      NaN        NaN             1900   
4    manufacturing facilitation efficiency      ab380  20/03/2021|1730   
..                                     ...        ...              ...   
122                                   MTBF   EAGLE AR              NaN   
123                                    oee  IDEALMOLD   monday|tuesday   
124                                    tst       ms90              NaN   
125                                   tsdt      ps410              NaN   
126                                    tpt     tlb139              NaN   

             <DATE> <TIME>     <DESC> <DIRECTION> <MEASUREMENT>  <QUANTITY>  
0        20/08/2030   2030        NaN         NaN           NaN         NaN  
1               NaN   1500        NaN         NaN           NaN         NaN  
2             today    830        NaN         NaN           NaN         NaN  
3               NaN   1900        NaN         NaN           NaN         NaN  
4        20/03/2021   1730        NaN         NaN           NaN         NaN  
..              ...    ...        ...         ...           ...         ...  
122             NaN    NaN        NaN    increase           NaN         NaN  
123  monday|tuesday    NaN  more than    increase           20%         NaN  
124             NaN    NaN  less than         NaN         30min         NaN  
125             NaN    NaN  more than         NaN         20min         NaN  
126             NaN    NaN  more than         NaN        2hours         NaN  

[127 rows x 11 columns]

In [4]:
pd.set_option('display.max_rows', None)

# Clean data

In [5]:
df_training = df_training.replace(np.nan, '')
df_training

22 june   
27                                    mtba      GW APP       last 2 hours   
28                   total productive time     LOTUS-E        5 hours ago   
29                       number of assists     is600gs       last 30 mins   
30                                    mtba     AEROLED                      
31                                     oee       BOT10                      
32                                     UPH        HiPO       last 4 hours   
33                                failures       hp229          yesterday   
34                                     UPH   aeroscope          yesterday   
35                  total engineering time       DS209          last week   
36                                     mfe       iHAWK          yesterday   
37                                     tet     is600gs          yesterday   
38                                    TUDT     is600gs                      
39                      number of failures        ms60          yesterday   
40                                     OEE        AD90              today   
41                                     oee        ad90                      
42                                    TNDT       RO832          last week   
43                                     UPH      ihb139          yesterday   
44                                     OEE      SD832D              today   
45                                  output       stk40                      
46                                     oee       stk40          yesterday   
47                                     oee       ab380          last week   
48                                     uph        AD90                      
49                                     oee       ab380                      
50                                  output     3rdbase                      
51               mean time between assists       AB350                      
52                   total productive time       AB380                      
53                  total engineering time       AB339                      
54            total nonscheduled down time       ab383                      
55                      total standby time       ab339                      
56                  total engineering time  ab350vlamp                      
57               mean time between failure  ab350vlamp                      
58   manufacturing facilitation efficiency       ab339                      
59                                     mfe     3rdbase                      
60                                     tpt     ab350vm                      
61                                     UPH     ab350vs                      
62            total nonscheduled down time     ab383dh                      
63                   total productive time     ab383dh                      
64               total scheduled down time       ad210                      
65                                  output   ab280plus                      
66                      number of failures       ad280                      
67                       number of assists       ab380                      
68                           unit per hour     ab383dh                      
69                                     tpt  ab350vlamp                      
70                                     tet   ad210plus                      
71                                    mtba       ad211                      
72                                    tudt        ad50                      
73                                     tst     ab350vs                      
74                                    mtbf     3rdbase                      
75                                     mfe   2d211plus                      
76                                  output       ad280                      
77                      number of failures     ab383dh                      
78                                    tsdt     ab350vm        

# Helper methods to get the index of each time of entity
## This step is done to prepare the training data 

In [6]:
def get_ATTR(Query, ATTR):

    start = 0
    end = 0
    attr_to_find = r'\b{}\b'.format(ATTR.lower())

    match = re.search(attr_to_find, Query.lower())

    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<ATTR>")

        return ent_tuple



def get_TEMP(Query, TEMP):

    start = 0
    end = 0
    temp_to_find = r'\b{}\b'.format(TEMP.lower())

    match = re.search(temp_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<TEMP>")
        # print(ent_tuple)
        return ent_tuple



def get_OBJ(Query, OBJ):
    start = 0
    end = 0
    obj_to_find = r'\b{}\b'.format(OBJ.lower())

    match = re.search(obj_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<OBJ>")
        return ent_tuple

def get_DESC(Query, OBJ):
    start = 0
    end = 0
    desc_to_find = r'\b{}\b'.format(OBJ.lower())

    match = re.search(desc_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<DESC>")
        return ent_tuple

def get_DIR(Query, OBJ):
    start = 0
    end = 0
    dir_to_find = r'\b{}\b'.format(OBJ.lower())

    match = re.search(dir_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<DIRECTION>")
        return ent_tuple

def get_MEASUREMENT(Query, MEASUREMENT):
    start = 0
    end = 0
    measurement_to_find = r'\b{}\b'.format(MEASUREMENT.lower())

    match = re.search(measurement_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<MEASUREMENT>")
        return ent_tuple

def get_QUANTITY(Query, QUANTITY):
    start = 0
    end = 0
    quantity_to_find = r'\b{}\b'.format(str(math.trunc(int(float(QUANTITY.lower())))))


    match = re.search(quantity_to_find, Query.lower())
    if match:
        start = match.start()
        end = match.end()
        ent_tuple = (start, end, "<QUANTITY>")
        return ent_tuple
# def get_DATE():

# def get_TIME():


def get_ATTR_OBJ_TEMP_DESC(Query, ATTR, OBJ, TEMP, DESC, MEASUREMENT):
    ent_list = []

    if len(ATTR) != 0:
        for a in ATTR:
            ent_list.append(get_ATTR(Query, a))
    
    for t in TEMP:   
        ent_list.append(get_TEMP(Query, t))
    for o in OBJ:
        ent_list.append(get_OBJ(Query, o))
    for d in DESC:
        ent_list.append(get_DESC(Query,d))
    for m in MEASUREMENT:
        ent_list.append(get_MEASUREMENT(Query,m))
    for q in QUANTITY:
        ent_list.append(get_QUANTITY(Query,q))

    return ent_list






# Build training data from the dataframe by calling helper methods

In [12]:
training_data = []

for i in df_training.index:
    ent = []

    Query = df_training['Query'][i]
    Query = Query.replace(u'\xa0', u' ')


    ATTR = str(df_training['<ATTR>'][i])
    if ATTR != '':
        ATTR = ATTR.replace(u'\xa0', u' ')
        list_of_attr = ATTR.split('|')
        for a in list_of_attr:
            ent.append(get_ATTR(Query, a))

    OBJ = str(df_training['<OBJ>'][i])
    if OBJ != '':
        OBJ = OBJ.replace(u'\xa0', u' ')
        list_of_obj = OBJ.split('|')
        for o in list_of_obj:
            ent.append(get_OBJ(Query, o))

    TEMP = str(df_training['<TEMPORAL>'][i])
    if TEMP != '':
        TEMP = TEMP.replace(u'\xa0', u' ')
        list_of_temp = TEMP.split('|')
        for t in list_of_temp:
            ent.append(get_TEMP(Query, t))

    DESC = str(df_training['<DESC>'][i])
    if DESC != '':
        DESC = DESC.replace(u'\xa0', u' ')
        list_of_desc = DESC.split('|')
        for d in list_of_desc:
            ent.append(get_DESC(Query, d))

    DIRECTION = str(df_training['<DIRECTION>'][i])
    if DIRECTION != '':
        DIRECTION = DIRECTION.replace(u'\xa0', u' ')
        list_of_dir = DIRECTION.split('|')
        for dir in list_of_dir:
            ent.append(get_DIR(Query, dir))
    
    MEASUREMENT = str(df_training['<MEASUREMENT>'][i])
    if MEASUREMENT != '':
        MEASUREMENT = MEASUREMENT.replace(u'\xa0', u' ')
        list_of_measurement = MEASUREMENT.split('|')
        for m in list_of_measurement:
            ent.append(get_MEASUREMENT(Query, m))

    QUANTITY = str(df_training['<QUANTITY>'][i])
    if QUANTITY != '':
        QUANTITY = QUANTITY.replace(u'\xa0', u' ')
        list_of_quantity = QUANTITY.split('|')
        for q in list_of_quantity:
            ent.append(get_QUANTITY(Query, q))

    ent_dict = {"entities": ent}
    training_data.append((Query, ent_dict))
    
    # TEMP = str(df_training['<TEMPORAL>'][i])
    # OBJ = str(df_training['<OBJ>'][i])
    # DATE = str(df_training['<DATE>'][i])
    # TIME = str(df_training['<TIME>'][i])
for i in training_data:
    print(i)

('Time is 2030 and the date is 20/08/2030', {'entities': [(8, 12, '<TEMP>'), (29, 39, '<TEMP>')]})
('Set a reminder at 1500 later', {'entities': [(18, 22, '<TEMP>')]})
('Today at 0830', {'entities': [(9, 13, '<TEMP>'), (0, 5, '<TEMP>')]})
('Print report at 1900', {'entities': [(16, 20, '<TEMP>')]})
('manufacturing facilitation efficiency of ab380 on 20/03/2021 at 1730', {'entities': [(0, 37, '<ATTR>'), (41, 46, '<OBJ>'), (50, 60, '<TEMP>'), (64, 68, '<TEMP>')]})
('What is the total engineering time of EAGLE60 on 20/06/1997', {'entities': [(12, 34, '<ATTR>'), (38, 45, '<OBJ>'), (49, 59, '<TEMP>')]})
('Current time is 12:00', {'entities': [(16, 21, '<TEMP>')]})
('Meet me at 23:20', {'entities': [(11, 16, '<TEMP>')]})
('I will be free at 17:40', {'entities': [(18, 23, '<TEMP>')]})
('2 Feb 2021', {'entities': [(0, 10, '<TEMP>')]})
('2 August 2023', {'entities': [(0, 13, '<TEMP>')]})
('2 September 2021', {'entities': [(0, 16, '<TEMP>')]})
('20 Mar', {'entities': [(0, 6, '<TEMP>')]})
('June 

# NOTE: THERE MIGHT BE FORMATTING ISSUES WITH % SIGNS
# I had to manually edit the 'NONE' tags to '<MEASUREMENT>' tags
## Below is the final training data

In [13]:
TRAINING_DATA = [
    ('Time is 2030 and the date is 20/08/2030', {'entities': [(8, 12, '<TEMP>'), (29, 39, '<TEMP>')]}),
    ('Set a reminder at 1500 later', {'entities': [(18, 22, '<TEMP>')]}),
    ('Today at 0830', {'entities': [(9, 13, '<TEMP>'), (0, 5, '<TEMP>')]}),
    ('Print report at 1900', {'entities': [(16, 20, '<TEMP>')]}),
    ('manufacturing facilitation efficiency of ab380 on 20/03/2021 at 1730', {'entities': [(0, 37, '<ATTR>'), (41, 46, '<OBJ>'), (50, 60, '<TEMP>'), (64, 68, '<TEMP>')]}),
    ('What is the total engineering time of EAGLE60 on 20/06/1997', {'entities': [(12, 34, '<ATTR>'), (38, 45, '<OBJ>'), (49, 59, '<TEMP>')]}),
    ('Current time is 12:00', {'entities': [(16, 21, '<TEMP>')]}),
    ('Meet me at 23:20', {'entities': [(11, 16, '<TEMP>')]}),
    ('I will be free at 17:40', {'entities': [(18, 23, '<TEMP>')]}),
    ('2 Feb 2021', {'entities': [(0, 10, '<TEMP>')]}),
    ('2 August 2023', {'entities': [(0, 13, '<TEMP>')]}),
    ('2 September 2021', {'entities': [(0, 16, '<TEMP>')]}),
    ('20 Mar', {'entities': [(0, 6, '<TEMP>')]}),
    ('June 22', {'entities': [(0, 7, '<TEMP>')]}),
    ('today is 21 Dec', {'entities': [(0, 5, '<TEMP>'), (9, 15, '<TEMP>')]}),
    ('2003 May 8', {'entities': [(0, 10, '<TEMP>')]}),
    ('0930 on 2003 April 4', {'entities': [(8, 20, '<TEMP>'), (0, 4, '<TEMP>')]}),
    ('Jun 22', {'entities': [(0, 6, '<TEMP>')]}),
    ('What is output from 11/03 to 11/04', {'entities': [(8, 14, '<ATTR>'), (20, 25, '<TEMP>'), (29, 34, '<TEMP>')]}),
    ('last week', {'entities': [(0, 9, '<TEMP>')]}),
    ('last month', {'entities': [(0, 10, '<TEMP>')]}),
    ('what is the date last monday', {'entities': [(17, 28, '<TEMP>')]}),
    ('output of AB350 yesterday', {'entities': [(0, 6, '<ATTR>'), (10, 15, '<OBJ>'), (16, 25, '<TEMP>')]}),
    ('MTBA of FT2030S last week', {'entities': [(0, 4, '<ATTR>'), (8, 15, '<OBJ>'), (16, 25, '<TEMP>')]}),
    ('what is TUDT of HP229 on 1 may', {'entities': [(8, 12, '<ATTR>'), (16, 21, '<OBJ>'), (25, 30, '<TEMP>')]}),
    ('total standby time of IHAWK yesterday', {'entities': [(0, 18, '<ATTR>'), (22, 27, '<OBJ>'), (28, 37, '<TEMP>')]}),
    ('22 june number of assists for ISLINDA', {'entities': [(8, 25, '<ATTR>'), (30, 37, '<OBJ>'), (0, 7, '<TEMP>')]}),
    ('mtba for GW APP in the last 2 hours', {'entities': [(0, 4, '<ATTR>'), (9, 15, '<OBJ>'), (23, 35, '<TEMP>')]}),
    ('total productive time of LOTUS-E 5 hours ago', {'entities': [(0, 21, '<ATTR>'), (25, 32, '<OBJ>'), (33, 44, '<TEMP>')]}),
    ('Number of assists in the last 30 mins for is600gs', {'entities': [(0, 17, '<ATTR>'), (42, 49, '<OBJ>'), (25, 37, '<TEMP>')]}),
    ('what is the increase in mtba for AEROLED', {'entities': [(24, 28, '<ATTR>'), (33, 40, '<OBJ>'), (12, 20, '<DIRECTION>')]}),
    ('what is the decrease in oee for BOT10', {'entities': [(24, 27, '<ATTR>'), (32, 37, '<OBJ>'), (12, 20, '<DIRECTION>')]}),
    ('increase in UPH for HiPO in the last 4 hours', {'entities': [(12, 15, '<ATTR>'), (20, 24, '<OBJ>'), (32, 44, '<TEMP>')]}),
    ('reduction in failures for hp229 yesterday', {'entities': [(13, 21, '<ATTR>'), (26, 31, '<OBJ>'), (32, 41, '<TEMP>')]}),
    ('lowest UPH for aeroscope yesterday', {'entities': [(7, 10, '<ATTR>'), (15, 24, '<OBJ>'), (25, 34, '<TEMP>'), (0, 6, '<DESC>')]}),
    ('DS209 highest total engineering time last week', {'entities': [(14, 36, '<ATTR>'), (0, 5, '<OBJ>'), (37, 46, '<TEMP>'), (6, 13, '<DESC>')]}),
    ('highest mfe yesterday for iHAWK', {'entities': [(8, 11, '<ATTR>'), (26, 31, '<OBJ>'), (12, 21, '<TEMP>'), (0, 7, '<DESC>')]}),
    ('which is600gs has the lowest tet yesterday', {'entities': [(29, 32, '<ATTR>'), (6, 13, '<OBJ>'), (33, 42, '<TEMP>')]}),
    ('what is the number of is600gs with low TUDT', {'entities': [(39, 43, '<ATTR>'), (22, 29, '<OBJ>')]}),
    ('greatest number of failures from ms60 yesterday', {'entities': [(9, 27, '<ATTR>'), (33, 37, '<OBJ>'), (38, 47, '<TEMP>'), (0, 8, '<DESC>')]}),
    ('why reduction in OEE in AD90 today', {'entities': [(17, 20, '<ATTR>'), (24, 28, '<OBJ>'), (29, 34, '<TEMP>'), (4, 13, '<DIRECTION>')]}),
    ('how to increase oee for ad90', {'entities': [(16, 19, '<ATTR>'), (24, 28, '<OBJ>'), (7, 15, '<DIRECTION>')]}),
    ('what is the decrease in TNDT RO832 last week', {'entities': [(24, 28, '<ATTR>'), (29, 34, '<OBJ>'), (35, 44, '<TEMP>'), (12, 20, '<DIRECTION>')]}),
    ('drop in uph of ihb139 yesterday', {'entities': [(8, 11, '<ATTR>'), (15, 21, '<OBJ>'), (22, 31, '<TEMP>'), (0, 4, '<DIRECTION>')]}),
    ('why OEE decrease for SD832D today', {'entities': [(4, 7, '<ATTR>'), (21, 27, '<OBJ>'), (28, 33, '<TEMP>'), (8, 16, '<DIRECTION>')]}),
    ('what caused output of stk40 to fall', {'entities': [(12, 18, '<ATTR>'), (22, 27, '<OBJ>'), (31, 35, '<DIRECTION>')]}),
    ('reason for reduction in oee for stk40 yesterday', {'entities': [(24, 27, '<ATTR>'), (32, 37, '<OBJ>'), (38, 47, '<TEMP>'), (11, 20, '<DIRECTION>')]}),
    ('how much did oee increase for ab380 last week', {'entities': [(13, 16, '<ATTR>'), (30, 35, '<OBJ>'), (36, 45, '<TEMP>'), (17, 25, '<DIRECTION>')]}),
    ('why did uph fall for AD90', {'entities': [(8, 11, '<ATTR>'), (21, 25, '<OBJ>'), (12, 16, '<DIRECTION>')]}),
    ('which ab380 has the lowest oee', {'entities': [(27, 30, '<ATTR>'), (6, 11, '<OBJ>'), (20, 26, '<DESC>')]}),
    ('What is output of 3rdbase', {'entities': [(8, 14, '<ATTR>'), (18, 25, '<OBJ>')]}),
    ('mean time between assists AB350', {'entities': [(0, 25, '<ATTR>'), (26, 31, '<OBJ>')]}),
    ('AB380 total productive time', {'entities': [(6, 27, '<ATTR>'), (0, 5, '<OBJ>')]}),
    ('total engineering time AB339', {'entities': [(0, 22, '<ATTR>'), (23, 28, '<OBJ>')]}),
    ('total nonscheduled down time ab383', {'entities': [(0, 28, '<ATTR>'), (29, 34, '<OBJ>')]}),
    ('total standby time ab339', {'entities': [(0, 18, '<ATTR>'), (19, 24, '<OBJ>')]}),
    ('ab350vlamp total engineering time', {'entities': [(11, 33, '<ATTR>'), (0, 10, '<OBJ>')]}),
    ('mean time between failure ab350vlamp', {'entities': [(0, 25, '<ATTR>'), (26, 36, '<OBJ>')]}),
    ('ab339 manufacturing facilitation efficiency', {'entities': [(6, 43, '<ATTR>'), (0, 5, '<OBJ>')]}),
    ('mfe of 3rdbase', {'entities': [(0, 3, '<ATTR>'), (7, 14, '<OBJ>')]}),
    ('tpt of ab350vm', {'entities': [(0, 3, '<ATTR>'), (7, 14, '<OBJ>')]}),
    ('what is the UPH of ab350vs', {'entities': [(12, 15, '<ATTR>'), (19, 26, '<OBJ>')]}),
    ('total nonscheduled down time of ab383dh', {'entities': [(0, 28, '<ATTR>'), (32, 39, '<OBJ>')]}),
    ('total productive time ab383dh', {'entities': [(0, 21, '<ATTR>'), (22, 29, '<OBJ>')]}),
    ('ad210 total scheduled down time', {'entities': [(6, 31, '<ATTR>'), (0, 5, '<OBJ>')]}),
    ('total output ab280plus', {'entities': [(6, 12, '<ATTR>'), (13, 22, '<OBJ>')]}),
    ('ad280 number of failures', {'entities': [(6, 24, '<ATTR>'), (0, 5, '<OBJ>')]}),
    ('ab380 number of assists', {'entities': [(6, 23, '<ATTR>'), (0, 5, '<OBJ>')]}),
    ('ab383dh unit per hour', {'entities': [(8, 21, '<ATTR>'), (0, 7, '<OBJ>')]}),
    ('ab350vlamp tpt', {'entities': [(11, 14, '<ATTR>'), (0, 10, '<OBJ>')]}),
    ('ad210plus tet', {'entities': [(10, 13, '<ATTR>'), (0, 9, '<OBJ>')]}),
    ('mtba of ad211', {'entities': [(0, 4, '<ATTR>'), (8, 13, '<OBJ>')]}),
    ('what is the tudt of ad50', {'entities': [(12, 16, '<ATTR>'), (20, 24, '<OBJ>')]}),
    ('tst ab350vs', {'entities': [(0, 3, '<ATTR>'), (4, 11, '<OBJ>')]}),
    ('3rdbase mtbf', {'entities': [(8, 12, '<ATTR>'), (0, 7, '<OBJ>')]}),
    ('mfe of 2d211plus', {'entities': [(0, 3, '<ATTR>'), (7, 16, '<OBJ>')]}),
    ('ad280 output', {'entities': [(6, 12, '<ATTR>'), (0, 5, '<OBJ>')]}),
    ('number of failures of ab383dh', {'entities': [(0, 18, '<ATTR>'), (22, 29, '<OBJ>')]}),
    ('what is the tsdt of ab350vm', {'entities': [(12, 16, '<ATTR>'), (20, 27, '<OBJ>')]}),
    ('tet of ad8312fc', {'entities': [(0, 3, '<ATTR>'), (7, 15, '<OBJ>')]}),
    ('what is the mfe of ad50plus', {'entities': [(12, 15, '<ATTR>'), (19, 27, '<OBJ>')]}),
    ('mean time between assists ad828', {'entities': [(0, 25, '<ATTR>'), (26, 31, '<OBJ>')]}),
    ('number of failures of ad830', {'entities': [(0, 18, '<ATTR>'), (22, 27, '<OBJ>')]}),
    ('output of ad8312', {'entities': [(0, 6, '<ATTR>'), (10, 16, '<OBJ>')]}),
    ('ad830l total unscheduled down time', {'entities': [(7, 34, '<ATTR>'), (0, 6, '<OBJ>')]}),
    ('ad830r total productive time', {'entities': [(7, 28, '<ATTR>'), (0, 6, '<OBJ>')]}),
    ('ad8312fc mtbf this monday', {'entities': [(9, 13, '<ATTR>'), (0, 8, '<OBJ>'), (14, 25, '<TEMP>')]}),
    ('what is the tndt for ab280', {'entities': [(12, 16, '<ATTR>'), (21, 26, '<OBJ>')]}),
    ('what is the number of assists ad8312', {'entities': [(12, 29, '<ATTR>'), (30, 36, '<OBJ>')]}),
    ('assists for ad828', {'entities': [(0, 7, '<ATTR>'), (12, 17, '<OBJ>')]}),
    ('output of ad211plus', {'entities': [(0, 6, '<ATTR>'), (10, 19, '<OBJ>')]}),
    ('ad830u tst', {'entities': [(7, 10, '<ATTR>'), (0, 6, '<OBJ>')]}),
    ('total standby time ad830ur', {'entities': [(0, 18, '<ATTR>'), (19, 26, '<OBJ>')]}),
    ('what is the number of failures for ad838lplus', {'entities': [(12, 30, '<ATTR>'), (35, 45, '<OBJ>')]}),
    ('Total engineering time of ad868', {'entities': [(0, 22, '<ATTR>'), (26, 31, '<OBJ>')]}),
    ('ad8912sd total productive time', {'entities': [(9, 30, '<ATTR>'), (0, 8, '<OBJ>')]}),
    ('what is the tndt ad838p', {'entities': [(12, 16, '<ATTR>'), (17, 23, '<OBJ>')]}),
    ('VIM300 failures', {'entities': [(7, 15, '<ATTR>'), (0, 6, '<OBJ>')]}),
    ('mtbf of HARRIER XT', {'entities': [(0, 4, '<ATTR>'), (8, 18, '<OBJ>')]}),
    ('what is the greatest increase in BFMF for RO832 4 hours ago', {'entities': [(33, 37, '<ATTR>'), (42, 47, '<OBJ>'), (48, 59, '<TEMP>'), (12, 20, '<DESC>'), (21, 29, '<DIRECTION>')]}),
    ('top 5 ms100 with highest oee last monday', {'entities': [(25, 28, '<ATTR>'), (6, 11, '<OBJ>'), (29, 40, '<TEMP>'), (17, 24, '<DESC>'), (4, 5, '<QUANTITY>')]}),
    ('ORCAS with most BHCC today', {'entities': [(16, 20, '<ATTR>'), (0, 5, '<OBJ>'), (21, 26, '<TEMP>'), (11, 15, '<DESC>')]}),
    ('3 Skyhawk with most BUCT 2 hours ago', {'entities': [(20, 24, '<ATTR>'), (2, 9, '<OBJ>'), (25, 36, '<TEMP>'), (15, 19, '<DESC>'), (0, 1, '<QUANTITY>')]}),
    ('how many WINWB have more than 5 wire clamp operations last week', {'entities': [(32, 53, '<ATTR>'), (9, 14, '<OBJ>'), (54, 63, '<TEMP>'), (20, 29, '<DESC>'), (30, 31, '<QUANTITY>')]}),
    ('AD90 with the longest TUDT yesterday', {'entities': [(22, 26, '<ATTR>'), (0, 4, '<OBJ>'), (27, 36, '<TEMP>'), (14, 21, '<DESC>')]}),
    ('select top 5 ab380 with most number of failures last week', {'entities': [(29, 47, '<ATTR>'), (13, 18, '<OBJ>'), (48, 57, '<TEMP>'), (24, 28, '<DESC>'), (11, 12, '<QUANTITY>')]}),
    ('5 ab380 with tst more than 40min', {'entities': [(13, 16, '<ATTR>'), (2, 7, '<OBJ>'), (17, 26, '<DESC>'), (27, 32, '<MEASUREMENT>'), (0, 1, '<QUANTITY>')]}),
    ('all ad90 tndt more than 90min', {'entities': [(9, 13, '<ATTR>'), (4, 8, '<OBJ>'), (14, 23, '<DESC>'), (24, 29, '<MEASUREMENT>')]}),
    ('what is the lowest UPH for MC609H last week', {'entities': [(19, 22, '<ATTR>'), (27, 33, '<OBJ>'), (34, 43, '<TEMP>'), (12, 18, '<DESC>')]}),
    ('ISPLASMA OEE decrease by how much from 2 hours ago', {'entities': [(9, 12, '<ATTR>'), (0, 8, '<OBJ>'), (39, 50, '<TEMP>'), (13, 21, '<DIRECTION>')]}),
    ('how much did IS838LA3 tet increase last week', {'entities': [(22, 25, '<ATTR>'), (13, 21, '<OBJ>'), (35, 44, '<TEMP>'), (26, 34, '<DIRECTION>')]}),
    ('how much IDEALMOLD tpt fall from yesterday to today', {'entities': [(19, 22, '<ATTR>'), (9, 18, '<OBJ>'), (33, 42, '<TEMP>'), (46, 51, '<TEMP>')]}),
    ('which cb830 uph decrease by 20% today', {'entities': [(12, 15, '<ATTR>'), (6, 11, '<OBJ>'), (32, 37, '<TEMP>'), (16, 24, '<DIRECTION>'), (28, 31, '<MEASUREMENT>')]}),
    ('how much did mtbf drop for COG902 in the last 20 mins', {'entities': [(13, 17, '<ATTR>'), (27, 33, '<OBJ>'), (41, 53, '<TEMP>'), (18, 22, '<DIRECTION>')]}),
    ('5 CS8KAP with assists higher than 50%', {'entities': [(14, 21, '<ATTR>'), (2, 8, '<OBJ>'), (22, 33, '<DESC>'), (34, 37, '<MEASUREMENT>'), (0, 1, '<QUANTITY>')]}),
    ('which cm868s have tndt greater than 30mins', {'entities': [(18, 22, '<ATTR>'), (6, 12, '<OBJ>'), (23, 35, '<DESC>'), (36, 42, '<MEASUREMENT>')]}),
    ('number of aerocami that output less than 30%', {'entities': [(24, 30, '<ATTR>'), (10, 18, '<OBJ>'), (31, 40, '<DESC>'), (41, 44, '<MEASUREMENT>')]}),
    ('what is the reduction in oee of aerogauge on wednesday', {'entities': [(25, 28, '<ATTR>'), (32, 41, '<OBJ>'), (45, 54, '<TEMP>'), (12, 21, '<DIRECTION>')]}),
    ('drop in uph of ihb139 yesterday', {'entities': [(8, 11, '<ATTR>'), (15, 21, '<OBJ>'), (22, 31, '<TEMP>'), (0, 4, '<DIRECTION>')]}),
    ('how many ihb139 uph drop below 30', {'entities': [(16, 19, '<ATTR>'), (9, 15, '<OBJ>'), (20, 24, '<DIRECTION>'), (31, 33, '<MEASUREMENT>')]}),
    ('what is the increase in mtba of ab380', {'entities': [(24, 28, '<ATTR>'), (32, 37, '<OBJ>'), (12, 20, '<DIRECTION>')]}),
    ('change in mtbf of at420 yesterday', {'entities': [(10, 14, '<ATTR>'), (18, 23, '<OBJ>'), (24, 33, '<TEMP>'), (0, 6, '<DIRECTION>')]}),
    ('MTBF of EAGLE AR increase by how much', {'entities': [(0, 4, '<ATTR>'), (8, 16, '<OBJ>'), (17, 25, '<DIRECTION>')]}),
    ('which IDEALMOLD oee increase more than 20% from monday to tuesday', {'entities': [(16, 19, '<ATTR>'), (6, 15, '<OBJ>'), (48, 54, '<TEMP>'), (58, 65, '<TEMP>'), (29, 38, '<DESC>'), (20, 28, '<DIRECTION>'), (39, 42,'<MEASUREMENT>')]}),
    ('number of ms90 tst less than 30min', {'entities': [(15, 18, '<ATTR>'), (10, 14, '<OBJ>'), (19, 28, '<DESC>'), (29, 34, '<MEASUREMENT>')]}),
('number of ps410 with tsdt more than 20min', {'entities': [(21, 25, '<ATTR>'), (10, 15, '<OBJ>'), (26, 35, '<DESC>'), (36, 41, '<MEASUREMENT>')]}),
('number of tlb139 with tpt more than 2hours', {'entities': [(22, 25, '<ATTR>'), (10, 16, '<OBJ>'), (26, 35, '<DESC>'), (36, 42, '<MEASUREMENT>')]})
    
]


# Switch off other components in the spacy pipeline
# Leave the Named Entity Recognition component active for training

In [14]:
for _, annotations in TRAINING_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Start Training

In [12]:
# optimizer = nlp.initialize()
with nlp.disable_pipes(*unaffected_pipes):
    optimizer = nlp.create_optimizer()
    for iteration in range(15):
        random.shuffle(TRAINING_DATA)
        losses = {}
        
        # batches = minibatch(TRAINING_DATA, size=compounding(4.0, 32,1.001))
        for texts, annotations in TRAINING_DATA:
            # texts, annotations = zip(*batch)
            # print(texts)
            # print(annotations)

            doc = nlp.make_doc(texts)

            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example],
                drop=0.5,
                losses=losses,
                sgd=optimizer
            )
            print("losses", losses)

09381525190377}
losses {'ner': 22.763152228770412}
losses {'ner': 22.810740674091825}
losses {'ner': 22.810775977466722}
losses {'ner': 22.810775977466932}
losses {'ner': 23.184162094144252}
losses {'ner': 23.18416222173631}
losses {'ner': 26.936189275749992}
losses {'ner': 26.9363128192938}
losses {'ner': 26.938262009492696}
losses {'ner': 26.938446181715552}
losses {'ner': 28.909872783040527}
losses {'ner': 28.909884344764627}
losses {'ner': 31.006905927166816}
losses {'ner': 31.00690592717585}
losses {'ner': 31.010026954706515}
losses {'ner': 31.01002695472404}
losses {'ner': 33.3161757221954}
losses {'ner': 33.31619686268549}
losses {'ner': 33.316651381221234}
losses {'ner': 33.33998931877754}
losses {'ner': 33.33998946065534}
losses {'ner': 35.30015006695827}
losses {'ner': 35.33033371390262}
losses {'ner': 35.33033372180524}
losses {'ner': 35.359635510550454}
losses {'ner': 35.35963551459072}
losses {'ner': 35.359645214511424}
losses {'ner': 35.360414776884554}
losses {'ner': 35.

# Method to test the model
## Type 'quit' to escape

In [15]:
while True:
    tokens = []
    user_input = input("Enter your query:")
    if user_input == 'quit':
        break
    doc = nlp(user_input)
    # for ent in doc.ents:
    displacy.render(doc, style="ent")
    print(doc)
    for d in doc:
        print(d.text, d.ent_type_)


# Save model to file

In [14]:
# nlp.to_disk('TOKENIZER_NER_ATTR_OBJ_TEMP_DESC_DIR_MEASUREMENT_QUANTITY')